## Teemad ja nende märgendamine lausetele test50

Süntaksianalüüsi tulemusel saadi sagedusloend lauses nimeüksuste alluvateks (children) olevatest omadussõnalistest täienditest (amod, mis kirjeldavad nimeüksuse omadusi), nimeüksuste ülemuseks (parent) olevatest verbidest (kirjeldavad, mida nimeüksus lauses teeb) ning nende ülemuste alluvateks olevatest määrustest (advmod, mis kirjeldavad tegevuse viisi, kohta, aega vms). Järgnevalt tuleb jaotada laused klassidesse.

In [1]:
from estnltk import Text
from estnltk.taggers.system.rule_taggers import AmbiguousRuleset, Ruleset, StaticExtractionRule
from estnltk.taggers import PhraseTagger
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
import csv
from collections import Counter

### I sõnavara ja laused

Temaatiliste märksõnade valikul võib olla abiks süntaksianalüüsi teel saadud sõnaloendist teatava sagedusega sõnade välja sorteerimine, kuid lõplik märksõnade loend tuleb koostada käsitsi.

In [297]:
with open('verbs_amods_advmods_frequencies.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [298]:
len(rows)

1372

In [299]:
rows_more_common = []

In [300]:
for i in range(len(rows)):
    if i == 0:
        continue
    if int(rows[i]['freq']) > 1:
        rows_more_common.append(rows[i])

In [301]:
rows_more_common[:20]

[{'lemma': 'hea', 'freq': '4', 'value': 'amod'},
 {'lemma': 'peamine', 'freq': '2', 'value': 'amod'},
 {'lemma': 'kukkuma', 'freq': '10', 'value': 'verb'},
 {'lemma': 'juhtuma', 'freq': '2', 'value': 'verb'},
 {'lemma': 'tegema', 'freq': '57', 'value': 'verb'},
 {'lemma': 'saama', 'freq': '65', 'value': 'verb'},
 {'lemma': 'jätkama', 'freq': '111', 'value': 'verb'},
 {'lemma': 'minema', 'freq': '28', 'value': 'verb'},
 {'lemma': 'olema', 'freq': '33', 'value': 'verb'},
 {'lemma': 'küsima', 'freq': '5', 'value': 'verb'},
 {'lemma': 'lugema', 'freq': '6', 'value': 'verb'},
 {'lemma': 'vaatama', 'freq': '12', 'value': 'verb'},
 {'lemma': 'teadma', 'freq': '19', 'value': 'verb'},
 {'lemma': 'esinema', 'freq': '6', 'value': 'verb'},
 {'lemma': 'lootma', 'freq': '2', 'value': 'verb'},
 {'lemma': 'arvama', 'freq': '10', 'value': 'verb'},
 {'lemma': 'kandideerima', 'freq': '12', 'value': 'verb'},
 {'lemma': 'käima', 'freq': '15', 'value': 'verb'},
 {'lemma': 'ütlema', 'freq': '90', 'value': 'v

In [302]:
with open('thematic_keywords.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [303]:
print(rows)

[{'lemma': 'callable', 'class': 'string'}, {'lemma': "'vastanduma',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'võitlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'häbistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'vaidlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nokkima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'kandideerima',", 'class': 'valimised'}, {'lemma': "'valima',", 'class': 'valimised'}, {'lemma': "'võitma',", 'class': 'valimised'}, {'lemma': "'lubama',", 'class': 'valimised'}, {'lemma': "'hääletama',", 'class': 'valimised'}, {'lemma': "'valetama',", 'class': 'valetamine'}, {'lemma': "'varjama',", 'class': 'valetamine'}, {'lemma': "'mustama',", 'class': 'valetamine'}, {'lemma': "'kuritarvitama',", 'class': 'valetamine'}, {'lemma': "'juhtima',", 'class': 'valitsemine'}, {'lemma': "'otsustama',", 'class': 'valitsemine'}, {'lemma': "'kaasama',", 'class': 'valitsemine'}, {'lemma': "'mõjutama',", 'class': 'valitsemine'}

Andmebaasist loetakse süntaktilise analüüsi kihiga laused.

In [12]:
sentences = []
sentence_ids = []

In [13]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()

In [14]:
for row in cur.execute("SELECT ID, sentence FROM sentences_syntax_analysis"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    sentences.append(sentence)
    sentence_ids.append(sentence_id)

In [15]:
con.close()

### PhraseTagger märksõnade lausetele märkimiseks

In [16]:
vocabulary_file = 'thematic_keywords.csv'
ruleset = Ruleset()
ruleset.load(file_name=vocabulary_file, key_column='lemma')

In [17]:
phrasetagger = PhraseTagger(output_layer='keywords',
                      input_layer='morph_analysis',
                      input_attribute='lemma',
                      ruleset=ruleset,
                      output_attributes=['class'],
                      #decorator=decorator,
                      conflict_resolver='KEEP_MAXIMAL',
                      ignore_case=True)

In [18]:
# Lausetele märgitakse peale märksõnade kiht
for sentence in sentences:
    phrasetagger.tag(sentence)

In [19]:
sentences_with_keywords = []

for sentence in sentences:
    if len(sentence.keywords) > 0:
        sentences_with_keywords.append(sentence)

display(sentences_with_keywords[20])
display(sentences_with_keywords[20].keywords)
display(sentences_with_keywords[20].stanza_ensemble_syntax)

Text(text='Kes ja kuidas seal hääletuse igal juhul selle salajase hääletuse tulemusega siis valiti pressivaenlaseks president ilves ja mittelang kuigi ütlen ausalt lang oli selles nimekirjas.')

Layer(name='keywords', attributes=('class',), spans=SL[EnvelopingSpan(['valiti'], [{'class': 'valimised'}])])

Layer(name='stanza_ensemble_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('Kes', [{'id': 1, 'lemma': 'kes', 'upostag': 'P', 'xpostag': 'P', 'feats': {'pl': 'pl', 'nom': 'nom'}, 'head': 13, 'deprel': 'obj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('ja', [{'id': 2, 'lemma': 'ja', 'upostag': 'J', 'xpostag': 'J', 'feats': {'sub': 'sub', 'crd': 'crd'}, 'head': 3, 'deprel': 'cc', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('kuidas', [{'id': 3, 'lemma': 'kuidas', 'upostag': 'D', 'xpostag': 'D', 'feats': {}, 'head': 1, 'deprel': 'conj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('seal', [{'id': 4, 'lemma': 'seal', 'upostag': 'D', 'xpostag': 'D', 'feats': {}, 'head': 13, 'deprel': 'advmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('hääletuse', [{'id': 5, 'lemma': 'hääletus', 'upostag': 'S', 'xpostag': 'S', 'feats': {'com': 'com', 'sg': 'sg', 'gen': 'gen'}, 'head': 13, 'deprel': 'obj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('igal', [{'id': 6, 'lemma': 'iga', 'upostag': 'P', 'xpostag': 'P', 'feats': {'sg': 'sg', 'ad': 'ad'}, 'head': 7, 'deprel': 'det', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('juhul', [{'id': 7, 'lemma': 'juht', 'upostag': 'S', 'xpostag': 'S', 'feats': {'com': 'com', 'sg': 'sg', 'ad': 'ad'}, 'head': 13, 'deprel': 'obl', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('selle', [{'id': 8, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': {'sg': 'sg', 'gen': 'gen'}, 'head': 10, 'deprel': 'det', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('salajase', [{'id': 9, 'lemma': 'salajane', 'upostag': 'A', 'xpostag': 'A', 'feats': {'pos': 'pos', 'sg': 'sg', 'gen': 'gen'}, 'head': 10, 'deprel': 'amod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('hääletuse', [{'id': 10, 'lemma': 'hääletus', 'upostag': 'S', 'xpostag': 'S', 'feats': {'com': 'com', 'sg': 'sg', 'gen': 'gen'}, 'head': 11, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('tulemusega', [{'id': 11, 'lemma': 'tulemus', 'upostag': 'S', 'xpostag': 'S', 'feats': {'com': 'com', 'sg': 'sg', 'kom': 'kom'}, 'head': 13, 'deprel': 'obl', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('siis', [{'id': 12, 'lemma': 'siis', 'upostag': 'D', 'xpostag': 'D', 'feats': {}, 'head': 13, 'deprel': 'advmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('valiti', [{'id': 13, 'lemma': 'valima', 'upostag': 'V', 'xpostag': 'V', 'feats': {'aux': 'aux', 'indic': 'indic', 'impf': 'impf', 'imps': 'imps', 'af': 'af'}, 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]),
Span('pressivaenlaseks', [{'id': 14, 'lemma': 'pressivaenlane', 'upostag': 'S', 'xpostag': 'S', 'feats': {'com': 'com', 'sg': 'sg', 'tr': 'tr'}, 'head': 13, 'deprel': 'xcomp', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('president', [{'id': 15, 'lemma': 'president', 'upostag': 'S', 'xpostag': 'S', 'feats': {'com': 'com', 'sg': 'sg', 'nom': 'nom'}, 'head': 13, 'deprel': 'obj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('ilves', [{'id': 16, 'lemma': 'il

Kuigi märksõna lausest leiti, ei pruugi see olla lauses seotud huvipakkuva isiku/organisatsiooniga. Seetõttu tuleb võtta arvesse ka süntaksianalüüsi kihti, et teha kindlaks, kas leitud märksõna esineb lauses seoses isiku/organisatsiooniga (kas täiendi, verbi või määruse positsioonil).

In [316]:
phrase = []

In [317]:
for entity in sentences_with_keywords[20].entity_phrases:
    for span in entity:
        phrase.append(span)

In [318]:
print(phrase)

[Span('ilves', [{'normalized_text': 'ilves', 'lemma': 'ilves', 'root': 'ilves', 'root_tokens': ['ilves'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}])]


In [319]:
keywrds = []

In [320]:
for word in sentences_with_keywords[20].keywords:
    keywrds.append(word[0])

In [321]:
print(keywrds)

[Span('valiti', [{'normalized_text': 'valiti', 'lemma': 'valima', 'root': 'vali', 'root_tokens': ['vali'], 'ending': 'ti', 'clitic': '', 'form': 'ti', 'partofspeech': 'V'}])]


In [323]:
for subspan in phrase:
    parent = sentences_with_keywords[20].stanza_ensemble_syntax.get(subspan)['parent_span']
    parent_word = sentences_with_keywords[20]['words'].get(parent) 
    if parent_word in phrase:
        continue
    parent_morph = sentences_with_keywords[20].morph_analysis.get(parent)
    if 'V' not in parent_morph['partofspeech']:
        continue
    print(parent['lemma'])
    print('Kas on ülemus: ', parent_morph in keywrds)
    
    # Verbist ülemuse alluv, kui on määrus
    parent_children = sentences_with_keywords[20].stanza_ensemble_syntax.get(parent)['children']
    if len(parent_children) > 1:
        for child in parent_children:
            child_word = sentence['words'].get(child) 
            if child_word in phrase:
                continue
            elif 'advmod' != child['deprel']:
                continue
            if child == keywrds[0]:
                print(child['lemma'])
                print('Kas on ülemuse alluv: ', child == keywrds[0])
                break
            print(child['lemma'])
            print('Kas on ülemuse alluv: ', child == keywrds[0])
            
    children = sentences_with_keywords[20].stanza_ensemble_syntax.get(subspan)['children']
    if len(children) > 0:
        for child in children:
            child_word = sentences_with_keywords[20]['words'].get(child) 
            if child_word in phrase:
                continue
            elif 'amod' != child['deprel']:
                continue
            if child == keywrds[0]:
                print(child['lemma'])
                print('Kas on alluv: ', child == keywrds[0])
                break
            print(child['lemma'])
            print('Kas on alluv: ', child == keywrds[0])
    
    print(list(s.text for s in phrase))
    print(parent['lemma'])

Kõigil lausetel:

In [21]:
sentences_with_classes = []

In [22]:
for sentence in sentences_with_keywords:
    keywrds = []
    for word in sentence.keywords:
        keywrds.append(word[0])
        
    classes = []
    
    for entity in sentence.entity_phrases:
        phrase = []
        for span in entity:
            phrase.append(span)
    
        for subspan in phrase:
            parent = sentence.stanza_ensemble_syntax.get(subspan)['parent_span']
            if parent != None:
                # Verbist ülemus
                parent_morph = sentence.morph_analysis.get(parent)
                if 'V' not in parent_morph['partofspeech']:
                    continue
                for i in range(len(keywrds)):
                    if keywrds[i] == parent_morph:
                        cl = sentence['keywords'][i]['class']
                        classes.append(cl)
                # Verbist ülemuse alluv, kui on määrus
                parent_children = sentence.stanza_ensemble_syntax.get(parent)['children']
                if len(parent_children) > 1:
                    for child in parent_children:
                        if 'advmod' != child['deprel']:
                            continue
                        child_morph = sentence.morph_analysis.get(child)
                        for i in range(len(keywrds)):
                            if keywrds[i] == child_morph:
                                cl = sentence['keywords'][i]['class']
                                classes.append(cl)
                                
            
            # Täiendid
            children = sentence.stanza_ensemble_syntax.get(subspan)['children']
            if len(children) > 0:
                for child in children:
                    if 'amod' != child['deprel']:
                        continue
                    child_morph = sentence.morph_analysis.get(child)
                    for i in range(len(keywrds)):
                        if keywrds[i] == child_morph:
                            cl = sentence['keywords'][i]['class']
                            classes.append(cl)
                            
    if len(classes) > 0:
        sentences_with_classes.append(dict(zip(tuple([classes]), [sentence])))

TypeError: unhashable type: 'list'

In [292]:
len(sentences_with_classes)

72

In [293]:
display(sentences_with_classes)

[{('valimised',): Text(text='Teatas et ta kavatseb ka riigikokku kandideerida kevadel et kuidas ta hakkab seal töötama sotside fraktsioonis kus Eiki nestoriga kestev kas tema elust tõmbuksid linnast oma soojast peadest välja kangust.')},
 {('valimised',): Text(text='Mida ma aga olen kuulnud et tegelikult Eesti energia mingisugust igasugune New lubab selle põlevkivimassiivide tegutsemis ongi ainult antud uurimiseks aga kaevandamisluba hilisemaks ajaks ei ole.')},
 {('valitsemine',): Text(text='Noh ma arvan et ma ei reeda seda kitsas mõttes oli suurusjärgu võrra kõrgem ehk oleme õigel teel et siin küsimus ongi selles et seda on Eesti energia nüüd küll oled otsustanud ära ei lähe ostma mitte mingit vara kui selleks on ka põlevkivi ja kui me saame see väga odav me ei ole seda vaja leida just ainumaja kohta kuu ei taha.')},
 {('valitsemine',): Text(text='Air Balticu otsustas lõpetada äralennud Tartust Riiga viidates lennutülile lennujuhtide lennutüli Tartus ja see on üks selline suuremat so